In [6]:
import torch
import warnings

from my_code.mysionna.utils import expand_to_rank
from my_code.mysionna import PI
def deg_2_rad(x):
    r"""
    Convert degree to radian

    Input
    ------
        x : Tensor
            Angles in degree

    Output
    -------
        y : Tensor
            Angles ``x`` converted to radian
    """
    y = x * torch.tensor(PI/180.0)
    return y.to(x.dtype)
def one_ring_corr_mat(phi_deg, num_ant, d_h=0.5, sigma_phi_deg=15, dtype=torch.complex64):
    r"""Generate covariance matrices from the one-ring model.

    This function generates approximate covariance matrices for the
    so-called `one-ring` model (Eq. 2.24) [BHS2017]_. A uniform
    linear array (ULA) with uniform antenna spacing is assumed. The elements
    of the covariance matrices are computed as:

    .. math::
        \mathbf{R}_{\ell,m} =
              \exp\left( j2\pi d_\text{H} (\ell -m)\sin(\varphi) \right)
              \exp\left( -\frac{\sigma_\varphi^2}{2}
              \left( 2\pi d_\text{H}(\ell -m)\cos(\varphi) \right)^2 \right)

    for :math:`\ell,m = 1,\dots, M`, where :math:`M` is the number of antennas,
    :math:`\varphi` is the angle of arrival, :math:`d_\text{H}` is the antenna
    spacing in multiples of the wavelength,
    and :math:`\sigma^2_\varphi` is the angular standard deviation.

    Input
    -----
    phi_deg : [n_0, ..., n_k], tf.float
        A tensor of arbitrary rank containing azimuth angles (deg) of arrival.

    num_ant : int
        Number of antennas

    d_h : float
        Antenna spacing in multiples of the wavelength. Defaults to 0.5.

    sigma_phi_deg : float
        Angular standard deviation (deg). Defaults to 15 (deg). Values greater
        than 15 should not be used as the approximation becomes invalid.

    dtype : tf.complex64, tf.complex128
        The dtype of the output.

    Output
    ------
    R : [n_0, ..., n_k, num_ant, num_ant], `dtype`
        Tensor containing the covariance matrices of the desired dtype.
    """

    if sigma_phi_deg > 15:
        warnings.warn("sigma_phi_deg should be smaller than 15.")
    # get real_dtype    
    if dtype == torch.complex32:
        real_dtype = torch.float16
    elif dtype == torch.complex64:
        real_dtype = torch.float32
    elif dtype == torch.complex128:
        real_dtype = torch.float64
    else:
        raise TypeError("Not found comfortable type")
    # Convert all inputs to radians
    phi_deg = phi_deg.to(dtype=real_dtype)
    sigma_phi_deg = torch.tensor(sigma_phi_deg, dtype=real_dtype)
    phi = deg_2_rad(phi_deg)
    sigma_phi = deg_2_rad(sigma_phi_deg)

    # Add dimensions for broadcasting
    phi = phi.unsqueeze(-1)
    sigma_phi = sigma_phi.unsqueeze(-1)

    # Compute first column
    c = torch.tensor(2 * PI * d_h, dtype=real_dtype)
    d = c * torch.arange(0, num_ant, dtype=real_dtype)
    d = expand_to_rank(d, phi.dim(), 0)

    a = torch.complex(torch.zeros_like(d,dtype=real_dtype), d * torch.sin(phi))
    exp_a = torch.exp(a)  # First exponential term

    b = -0.5 * (sigma_phi * d * torch.cos(phi)) ** 2
    exp_b = torch.exp(b).to(dtype)  # Second exponential term

    col = exp_a * exp_b  # First column

    # First row is just the complex conjugate of first column
    row = torch.conj(col)

    # Create Toeplitz matrix using first column and first row
    toeplitz_matrix = torch.zeros((*col.shape[:-1], num_ant, num_ant), dtype=dtype)
    for i in range(num_ant):
        for j in range(num_ant):
            if i >= j:
                toeplitz_matrix[..., i, j] = col[..., i - j]
            else:
                toeplitz_matrix[..., i, j] = row[..., j - i]

    return toeplitz_matrix

# 测试例子
phi_deg = torch.tensor([30.0])
num_ant = 4
result = one_ring_corr_mat(phi_deg, num_ant)

print("Result:")
print(result)
print(result.shape)

In [18]:
import tensorflow as tf
import warnings

from sionna import PI
from sionna.utils import expand_to_rank
def deg_2_rad(x):
    r"""
    Convert degree to radian

    Input
    ------
        x : Tensor
            Angles in degree

    Output
    -------
        y : Tensor
            Angles ``x`` converted to radian
    """
    return x*tf.constant(PI/180.0, x.dtype)

def one_ring_corr_mat(phi_deg, num_ant, d_h=0.5, sigma_phi_deg=15,
                      dtype=tf.complex64):
    r"""Generate covariance matrices from the one-ring model.

    This function generates approximate covariance matrices for the
    so-called `one-ring` model (Eq. 2.24) [BHS2017]_. A uniform
    linear array (ULA) with uniform antenna spacing is assumed. The elements
    of the covariance matrices are computed as:

    .. math::
        \mathbf{R}_{\ell,m} =
              \exp\left( j2\pi d_\text{H} (\ell -m)\sin(\varphi) \right)
              \exp\left( -\frac{\sigma_\varphi^2}{2}
              \left( 2\pi d_\text{H}(\ell -m)\cos(\varphi) \right)^2 \right)

    for :math:`\ell,m = 1,\dots, M`, where :math:`M` is the number of antennas,
    :math:`\varphi` is the angle of arrival, :math:`d_\text{H}` is the antenna
    spacing in multiples of the wavelength,
    and :math:`\sigma^2_\varphi` is the angular standard deviation.

    Input
    -----
    phi_deg : [n_0, ..., n_k], tf.float
        A tensor of arbitrary rank containing azimuth angles (deg) of arrival.

    num_ant : int
        Number of antennas

    d_h : float
        Antenna spacing in multiples of the wavelength. Defaults to 0.5.

    sigma_phi_deg : float
        Angular standard deviation (deg). Defaults to 15 (deg). Values greater
        than 15 should not be used as the approximation becomes invalid.

    dtype : tf.complex64, tf.complex128
        The dtype of the output.

    Output
    ------
    R : [n_0, ..., n_k, num_ant, nun_ant], `dtype`
        Tensor containing the covariance matrices of the desired dtype.
    """

    if sigma_phi_deg>15:
        warnings.warn("sigma_phi_deg should be smaller than 15.")

    # Convert all inputs to radians
    phi_deg = tf.cast(phi_deg, dtype=dtype.real_dtype)
    sigma_phi_deg = tf.cast(sigma_phi_deg, dtype=dtype.real_dtype)
    phi = deg_2_rad(phi_deg)
    sigma_phi = deg_2_rad(sigma_phi_deg)

    # Add dimensions for broadcasting
    phi = tf.expand_dims(phi, -1)
    sigma_phi = tf.expand_dims(sigma_phi, -1)

    # Compute first column
    c = tf.constant(2*PI*d_h, dtype=dtype.real_dtype)
    d = c*tf.range(0, num_ant, dtype=dtype.real_dtype)
    d = expand_to_rank(d, tf.rank(phi), 0)

    a = tf.complex(tf.cast(0, dtype=dtype.real_dtype), d*tf.sin(phi))
    exp_a = tf.exp(a) # First exponential term

    b = -tf.cast(0.5, dtype=dtype.real_dtype)*(sigma_phi*d*tf.cos(phi))**2
    exp_b = tf.cast(tf.exp(b), dtype=dtype) # Second exponetial term

    col = exp_a*exp_b # First column

    # First row is just the complex conjugate of first column
    row = tf.math.conj(col)

    # Create Toeplitz operator
    operator = tf.linalg.LinearOperatorToeplitz(col, row)

    # Generate dense tensor from operator
    r = operator.to_dense()

    return r

# 测试例子
phi_deg = tf.constant([30.0])
num_ant = 4
result = one_ring_corr_mat(phi_deg, num_ant)

print("Result:")
print(result)

Result:
tf.Tensor(
[[[ 1.0000000e+00+0.0000000e+00j -3.3917775e-08-7.7594823e-01j
   -3.6251909e-01+3.1692426e-08j  1.2160416e-09+1.0197516e-01j]
  [-3.3917775e-08+7.7594823e-01j  1.0000000e+00+0.0000000e+00j
   -3.3917775e-08-7.7594823e-01j -3.6251909e-01+3.1692426e-08j]
  [-3.6251909e-01-3.1692426e-08j -3.3917775e-08+7.7594823e-01j
    1.0000000e+00+0.0000000e+00j -3.3917775e-08-7.7594823e-01j]
  [ 1.2160416e-09-1.0197516e-01j -3.6251909e-01-3.1692426e-08j
   -3.3917775e-08+7.7594823e-01j  1.0000000e+00+0.0000000e+00j]]], shape=(1, 4, 4), dtype=complex64)


In [7]:
try:
    import my_code
except ImportError as e:
    import sys
    sys.path.append("../")
from my_code.mysionna.channel.torch_version.utils import exp_corr_mat, one_ring_corr_mat
from my_code.mysionna.channel.torch_version.spatial_correlation import KroneckerModel,PerColumnModel
from my_code.mysionna.utils import complex_normal, matrix_sqrt

import pytest
import unittest
import numpy as np
import torch

from my_code.mysionna.utils.tensors import matrix_sqrt
# 获取所有可用的 GPU
gpus = torch.cuda.device_count()
print('Number of GPUs available:', gpus)

if gpus > 0:
    gpu_num = 0  # 要使用的 GPU 编号

    # 设置默认的 GPU
    torch.cuda.set_device(gpu_num)
    print('Only GPU number', gpu_num, 'used.')

    # 设置 GPU 内存增长模式
    device = torch.device(f'cuda:{gpu_num}')
    torch.cuda.empty_cache()
    try:
        torch.cuda.memory_allocated = 0
        torch.cuda.memory_reserved = 0
    except RuntimeError as e:
        print(e)

Number of GPUs available: 1
Only GPU number 0 used.


In [36]:
import tensorflow as tf

# 创建一个随机的复数矩阵
h = tf.constant([[[1+2j, 2+3j], [3+4j, 4+5j],[5+6j,6+7j]],[[1+2j, 2+3j], [3+4j, 4+5j],[5+6j,6+7j]]], dtype=tf.complex64)

# 计算共轭转置乘以自身
result = tf.matmul(h, h, adjoint_b=True)

print("TensorFlow result:")
print(result)


TensorFlow result:
tf.Tensor(
[[[ 18.+0.j  34.+4.j  50.+8.j]
  [ 34.-4.j  66.+0.j  98.+4.j]
  [ 50.-8.j  98.-4.j 146.+0.j]]

 [[ 18.+0.j  34.+4.j  50.+8.j]
  [ 34.-4.j  66.+0.j  98.+4.j]
  [ 50.-8.j  98.-4.j 146.+0.j]]], shape=(2, 3, 3), dtype=complex64)


In [37]:
import torch

# 创建一个随机的复数矩阵
h = torch.tensor([[[1+2j, 2+3j], [3+4j, 4+5j],[5+6j,6+7j]],[[1+2j, 2+3j], [3+4j, 4+5j],[5+6j,6+7j]]], dtype=torch.complex64)

# 计算共轭转置乘以自身
result = torch.matmul(h, h.transpose(-1, -2).conj())

print("PyTorch result:")
print(result)


PyTorch result:
tensor([[[ 18.+0.j,  34.+4.j,  50.+8.j],
         [ 34.-4.j,  66.+0.j,  98.+4.j],
         [ 50.-8.j,  98.-4.j, 146.+0.j]],

        [[ 18.+0.j,  34.+4.j,  50.+8.j],
         [ 34.-4.j,  66.+0.j,  98.+4.j],
         [ 50.-8.j,  98.-4.j, 146.+0.j]]])
